In [ ]:
import numpy as np
import netCDF4 as nc
import xarray as xr
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns

##### path declaration 

In [ ]:
station_path = '../input/iiitb-ai511-ams-solar-energy-prediction-contest/station_info.csv'
train_path = '../input/iiitb-ai511-ams-solar-energy-prediction-contest/train.csv'
test_path = '../input/iiitb-ai511-ams-solar-energy-prediction-contest/test.csv'
elevation_path = '../input/iiitb-ai511-ams-solar-energy-prediction-contest/gefs_elevations.nc'
train_weather_file_list = glob.glob('../input/iiitb-ai511-ams-solar-energy-prediction-contest/gefs_train_updated/train/*.nc')
test_weather_file_list = glob.glob('../input/iiitb-ai511-ams-solar-energy-prediction-contest/gefs_test_updated/test/*.nc')

## Preprocessing

5113*11*5*16*7

### helper functions

In [ ]:
def get_elevations_df(path):
  elevation = xr.open_dataset(elevation_path)
  df = elevation.to_dataframe().reset_index()
  df.drop(columns=["lat", "lon"], inplace=True)
  df.rename(columns={'latitude': 'nearest_lat', 'longitude': 'nearest_lon'}, inplace=True)
  return df

In [ ]:
stations = pd.read_csv(station_path)
train_data = pd.read_csv(train_path)
elevation = get_elevations_df(elevation_path)

In [ ]:
test_data = pd.read_csv(test_path)

In [ ]:
def stations_elevation_data_preprocessing(stations, elevation):
  stations["nearest_lat"] = np.NaN
  stations["nearest_lon"] = np.NaN
  stations["elevation_control"] = np.NaN
  stations["elevation_perturbation"] = np.NaN

  for index, row in stations.iterrows():
    lat, lon = closest_gridpoint(row["stid"])
    stations.at[index, "nearest_lat"] = lat
    stations.at[index, "nearest_lon"] = lon
    stations.at[index, "elevation_control"] = elevation.loc[(elevation.nearest_lat == lat) & (elevation.nearest_lon == lon),  "elevation_control"]
    stations.at[index, "elevation_perturbation"] = elevation.loc[(elevation.nearest_lat == lat) & (elevation.nearest_lon == lon),  "elevation_perturbation"]
 
  final = stations.copy()
  stations.drop(columns=["nearest_lat", "nearest_lon", "elevation_control", "elevation_perturbation"], inplace=True)
  return final

In [ ]:
def combine_stations_with_train_data(stations, train_data):
  train_data_melt = pd.melt(train_data, id_vars='Date', var_name='stid', value_name='energy')
  final = pd.merge(train_data_melt, stations)
  final['elon'] = final['elon'] + 360
  return final

In [ ]:
def date_preprocessing(df):
  date_list = df['Date'].unique()
  date_list.sort()
  final_dt = []
  
  for i, date in enumerate(date_list):
    temp = [i, date]
    final_dt.append(temp)

  date_df = pd.DataFrame(final_dt, columns=['date_id', 'Date'])
  df = pd.merge(df, date_df)

  df["year"] = df["Date"].astype("str").str.slice(0, 4)
  df["month"] = df["Date"].astype("str").str.slice(4, 6)
  df["day"] = df["Date"].astype("str").str.slice(6, 8)

  return df


In [ ]:
def closest_gridpoint(station_id):
    '''
    returns closest longitude latitude point
    '''
    st_lat = stations[stations.stid == station_id]['nlat'].iloc[0]
    st_lon = stations[stations.stid == station_id]['elon'].iloc[0] + 360

    lats = list(range(31,40))
    lons = list(range(254,270))
    
    lat_dif = list(abs(lats - st_lat))
    lon_dif = list(abs(lons - st_lon))
    
    return lats[lat_dif.index(min(lat_dif))], lons[lon_dif.index(min(lon_dif))]

In [ ]:
def predict_daily_mean(path, df_1, type):
  """
    calculates daily mean of forcast value at each station at particular date using its closest GEFS point.
  """
  const = 98

  df = df_1.copy()
  stations_id = df['stid'].unique()
  date_id = df['date_id'].unique()

  nc_dataset = nc.Dataset(path)
  w_variable_name = list(nc_dataset.variables.keys())[-1]

  df[w_variable_name] = np.NaN

  if(type == 0):
    for i, station_id in enumerate(stations_id):
      X = get_station_wise_data(path, station_id)

      for j, date in enumerate(date_id):
        date_wise_data = X[date, :, :]
        daily_mean_value = np.ma.mean(date_wise_data)
        # df.loc[(df.stid == station_id) & (df.date_id == date), w_variable_name] = daily_mean_value
        df.at[i + const*j , w_variable_name] = daily_mean_value
  else:
    for i, station_id in enumerate(stations_id):
      X = get_station_wise_data(path, station_id)

      for j, date in enumerate(date_id):
        date_wise_data = X[date, :, :]
        daily_mean_value = np.ma.mean(date_wise_data)
        df.loc[(df.stid == station_id) & (df.date_id == date), w_variable_name] = daily_mean_value
  return df

In [ ]:
def get_station_wise_data(path, stid):
  '''
      given station it returns data of particular station in 3d matrix form.
  '''
  lat, lon = closest_gridpoint(stid)
  nc_dataset = nc.Dataset(path, 'r')
  nc_dataset_variable_values = nc_dataset.variables.values()

  X_lat = list(nc_dataset_variable_values)[2]
  X_lon = list(nc_dataset_variable_values)[3]
  X_ind = []
  
  for i in range(len(X_lat)):
    for j in range(len(X_lon)):
      if (X_lat[i] == lat and X_lon[j] == lon):
        X_ind.append((i, j))

  X = []
  for i, j in X_ind:
    X.append(list(nc_dataset_variable_values)[-1][:, :, :, i, j])
  return X[0]

### Taking mean of daily forecast value by each 11 ensemble for all 15 forecast models



In [ ]:
def final_train_data_using_daily_mean_fun(weather_file_list, stations, elevation, train_data):
  df = stations_elevation_data_preprocessing(stations, elevation)
  df = combine_stations_with_train_data(df, train_data)
  df = date_preprocessing(df)
  final_data_using_daily_mean = df

  print("0/15...")
  for i, weather_file in enumerate(weather_file_list):
    df1 = predict_daily_mean(weather_file, df[['stid', 'date_id']], 0)
    print(str(i+1) + "/15...")
    final_data_using_daily_mean = final_data_using_daily_mean.merge(df1)
  
  final_data_using_daily_mean["Date"] = final_data_using_daily_mean["Date"].astype(str)
  final_data_using_daily_mean["Date"] = final_data_using_daily_mean["stid"] + "-" + final_data_using_daily_mean["Date"]
  final_data_using_daily_mean.drop(columns=["date_id", "nearest_lat", "nearest_lon"], inplace=True)

  return final_data_using_daily_mean

In [ ]:
def final_test_data_using_daily_mean_fun(weather_file_list, stations, elevation, test_path):
  test_df = pd.read_csv(test_path)
  test = test_df["Date"].str.split("-", n = 1, expand = True)
  test_df["Date"] = test[1]
  test_df["stid"] = test[0]
  df = test_df.merge(stations)
  df["elon"] = df["elon"] + 360

  df = date_preprocessing(df)
  st = stations_elevation_data_preprocessing(stations, elevation)
  df = pd.merge(df, st[["stid", "elevation_control", "elevation_perturbation"]], on="stid")
  df["date_id"] = df["date_id"]+4018
  final_data_using_daily_mean = df

  print("0/15...")
  for i, weather_file in enumerate(weather_file_list):
    df1 = predict_daily_mean(weather_file, df[['stid', 'date_id']], 1)
    print(str(i+1) + "/15...")
    final_data_using_daily_mean = final_data_using_daily_mean.merge(df1)
  
  final_data_using_daily_mean["Date"] = final_data_using_daily_mean["Date"].astype(str)
  final_data_using_daily_mean["Date"] = final_data_using_daily_mean["stid"] + "-" + final_data_using_daily_mean["Date"]
  final_data_using_daily_mean.drop(columns=["date_id"], inplace=True)

  final_data_using_daily_mean.to_csv('test_data.csv', index=False)
  return final_data_using_daily_mean

In [ ]:
%%time
train_data_using_daily_mean = final_train_data_using_daily_mean_fun(train_weather_file_list, stations, elevation, train_data)

In [ ]:
def generate_required_station_data(df):
  stations = ["HINT", "IDAB", "SLAP", "WEST", "BESS", "ACME"]
  final = pd.DataFrame(columns=list(df.columns))

  for station in stations:
    final = final.append(df[df.stid == station])

  final.to_csv("train_data.csv", index=False)

In [ ]:
generate_required_station_data(train_data_using_daily_mean)

In [ ]:
%%time
test_data_using_daily_mean = final_test_data_using_daily_mean_fun(train_weather_file_list, stations, elevation, test_path)

# Model Training


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
train = './train_data.csv'
test = './test_data.csv'

In [ ]:
df = pd.read_csv(train)

## LAbel encoding

In [ ]:
train_dates = df["Date"]
df.drop(columns="Date", inplace=True)

In [ ]:
le2 = preprocessing.LabelEncoder()
le2.fit(df["stid"].unique())
df["stid"] = le2.transform(df["stid"])

In [ ]:
X = df.loc[:, df.columns != 'energy'].to_numpy(dtype=np.float64)
y = df['energy'].to_numpy(dtype=np.float64)

## Training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state = 45)

### GBR

In [ ]:
reg_gbr = GradientBoostingRegressor(loss='lad', n_estimators=3000, max_features=24, learning_rate=0.035, max_depth=5, subsample=0.5, verbose=1)

## computation of training error
# reg_gbr.fit(X_train, y_train)
# preds = reg_gbr.predict(X_test)
# mae4 =  np.sqrt(metrics.mean_squared_error(y_test,preds))
# mae4

In [ ]:
reg_gbr.fit(X, y)

## Testing

In [ ]:
test_df = pd.read_csv(test)

In [ ]:
test_dates = test_df["Date"]
test_df.drop(columns="Date", inplace=True)
df_cols = list(df.columns)
df_cols.remove("energy")

In [ ]:
test_df = test_df[df_cols]
test_df["stid"] = le2.transform(test_df["stid"])

In [ ]:
Xt = test_df.loc[:, test_df.columns != 'energy'].to_numpy(dtype=np.float64)

In [ ]:
preds = reg_gbr.predict(Xt)

In [ ]:
submission_df = pd.DataFrame()
submission_df["Id"] = test_dates
submission_df["Expected"] = preds
submission_df.to_csv('final.csv', index=False)